In [1]:
import numpy as np
import cupy as cp

In [2]:
from numba import cuda

In [3]:
cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 4
                              PCI Bus ID: 0
                                    UUID: GPU-f17c582b-653b-1042-8b7f-e1b192c8f779
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

## Sending data to the GPU

In [4]:
array_cpu = np.random.randint(0, 255, size=(4000, 4000))

In [5]:
array_cpu.nbytes / 1e6

128.0

In [6]:
array_gpu = cp.asarray(array_cpu)


In [7]:
%%timeit

cp.asarray(array_cpu)

31.5 ms ± 3.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
type(array_gpu)

cupy.ndarray

In [9]:
from scipy import fft

In [10]:
%%timeit

fft.fftn(array_cpu)

536 ms ± 78.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
from cupyx.scipy import fft as fft_gpu

In [12]:
%%timeit

fft_gpu.fftn(array_gpu)

107 µs ± 46.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
fft_cpu = fft.fftn(array_cpu)
fft_sent_back = cp.asnumpy(fft_gpu.fftn(array_gpu))
np.allclose(fft_sent_back, fft_cpu)

True

In [14]:
# some numpy functions may work
np.max(array_gpu)

array(254)

In [15]:
%%time

# if at all possible, create arrays directly on the GPU
random_gpu_array = cp.random.randint(0, 255, size=(100, 100))

CPU times: user 1.37 s, sys: 27.8 ms, total: 1.4 s
Wall time: 1.61 s


## Custom kernels with numba

### Numba device arrays

In [16]:
# numba has its own API for transfering data
d_ary = cuda.to_device(array_cpu)
d_ary

In [17]:
cp.asarray(d_ary)

array([[175,  20, 105, ..., 142,  45,  89],
       [177, 144, 210, ..., 177, 214,  26],
       [108, 155,  22, ..., 125, 248,  24],
       ...,
       [ 81,  78, 149, ..., 136, 226, 136],
       [ 41,  83, 168, ..., 188, 221, 211],
       [134, 193, 149, ...,  43,  39, 156]])

In [18]:
d_ary.copy_to_host()

array([[175,  20, 105, ..., 142,  45,  89],
       [177, 144, 210, ..., 177, 214,  26],
       [108, 155,  22, ..., 125, 248,  24],
       ...,
       [ 81,  78, 149, ..., 136, 226, 136],
       [ 41,  83, 168, ..., 188, 221, 211],
       [134, 193, 149, ...,  43,  39, 156]])

### Numba kernels

In [19]:
from numba import cuda
import cupy as cp
import numpy as np

In [20]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:   # grid can extend beyond C
        tmp = 0.                            
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]        # multiply elements in row i of A and column j of B and add to temp
        C[i, j] = tmp

In [21]:
cp.random.seed(42)
A = cp.random.uniform(1, 10, size=(2000, 2000), dtype=np.float64)  # array 1
B = cp.random.uniform(1, 10, size=(2000, 2000), dtype=np.float64)  # array 2
C = cp.zeros((2000, 2000), dtype=np.float64)       # array where we store answer 

In [22]:
C

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
threadsperblock = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(C.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(C.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(125, 125)
The kernel will be executed up to element 2000


In [24]:
# execution of the kernel
matmul[blockspergrid, threadsperblock](A, B, C)

In [25]:
C

array([[59394.46607842, 58001.66377549, 58910.89964126, ...,
        58755.23643036, 59265.65525416, 58447.86197932],
       [59656.82462269, 58635.04995946, 59080.54393462, ...,
        59327.90030958, 60391.24930458, 59425.35827899],
       [62192.77335924, 60700.17680915, 60538.34933653, ...,
        61027.03460329, 61711.10155029, 60544.69882075],
       ...,
       [60649.27416407, 59951.20972379, 60170.2004206 , ...,
        60203.88074659, 60934.19598791, 59613.28418599],
       [61620.11922557, 61264.33868343, 62076.33462258, ...,
        61227.57661876, 62642.97523374, 61841.46799761],
       [61535.95697543, 59600.43760873, 59927.620961  , ...,
        60738.55627077, 61429.70009593, 59662.34901713]])

In [26]:
# faster multiplication can be obtained by making use of shared memory between threads in the same block
# this requires more thinking about non-obvious implementation!

from numba import float32, int32, float64

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [27]:
# execution of the kernel
SIZE = 2000
A = cp.random.uniform(1, 10, size=(SIZE, SIZE), dtype=np.float32)
B = cp.random.uniform(1, 10, size=(SIZE, SIZE), dtype=np.float32)
C_slow = cp.zeros((SIZE, SIZE), dtype=np.float32)      
C_fast = cp.zeros((SIZE, SIZE), dtype=np.float32)

In [28]:
threadsperblock = (TPB, TPB)
blockspergrid = int(np.ceil(SIZE / threadsperblock[0]))
blockspergrid = (blockspergrid, blockspergrid)

In [29]:
matmul[blockspergrid, threadsperblock](A, B, C_slow)

In [30]:
fast_matmul[blockspergrid, threadsperblock](A, B, C_fast)

In [31]:
cp.allclose(C_slow, C_fast)

array(True)

In [32]:
%%time

for i in range(10):
  matmul[blockspergrid, threadsperblock](A, B, C_slow)

CPU times: user 1.65 s, sys: 6.07 ms, total: 1.66 s
Wall time: 1.65 s


In [33]:
%%time

for i in range(10):
  fast_matmul[blockspergrid, threadsperblock](A, B, C_fast)

CPU times: user 1.53 s, sys: 0 ns, total: 1.53 s
Wall time: 1.52 s


In [34]:
C_c = cp.dot(A, B)

In [35]:
np.allclose(C_c, C_fast)

array(True)

In [36]:
%time

for i in range(10):
  cp.dot(A, B)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
